# Visual data analysis using STL decomposition

## Environment setup

For interactive plots

In [ ]:
%matplotlib notebook

Libraries

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from  sklearn import model_selection
import datetime
from tqdm import tqdm

import statsmodels.api as sm
from multiprocessing import Pool


## Data setup

Reading data from file

In [4]:
raw_data = pd.read_excel('2016-2020.xlsx')

Data processing
- processing time to EPOCH
- removing unlocalized data
- removing data without a proper type

In [5]:
data = raw_data

data = data[(data.Z < 0)]

data = data[data.TYP != '0']
data = data.assign(
    TIME = lambda dataframe: dataframe.apply(lambda x: datetime.datetime.fromisoformat(f'{x.DATA.replace(".","-")} {str(x.GODZ).rjust(2,"0")}:{str(x.MIN).rjust(2,"0")}:{str(x.SEK).rjust(2,"0")}').timestamp(),axis=1)
)

data = data.sort_values(by='TIME').reset_index(drop=True)


### Analisys

Function for getting time series of seismic activity from given event data

In [6]:
def get_activity_time_series(data,  t_min = None, t_max = None, dt=3600):

    if t_max is None:
        t_max = max(data.TIME)
    if t_min is None:
        t_min = min(data.TIME)

    t = pd.date_range(datetime.datetime.fromtimestamp(t_min),datetime.datetime.fromtimestamp(t_max), freq='1h')
    values = []
    i = 0
    for tt in tqdm(t):
        if i < len(data) and data.TIME[i] < tt.timestamp():
            values.append(data.ENG[i])
            i+=1
        else:
            values.append(0)
    
    return pd.Series(values, index=t)



Function for calculating STL decomposition and ploting it

In [7]:
def create_stl_plot(series,title=None):
    
    res = sm.tsa.seasonal_decompose(series, period=24*365, two_sided=True)
    fig, axs = plt.subplots(4,gridspec_kw=dict(hspace=0.5), figsize=(10, 10))
    
    for a in axs:
        a.set_xlim(min(series.keys()),max(series.keys()))
    
    observed = res.observed
    trend = res.trend
    seasonal = res.seasonal
    resid = res.resid
    
    axs[0].plot(observed,color='orange')
    
    axs[1].plot(trend.rolling(window=1250).mean(),color='green')
    
    axs[2].plot(seasonal)
    
    resid = resid.abs()
    
    axs[3].scatter(resid.keys(),resid.values, s=3, color='red' )
    
    axs[0].title.set_text('Energia wstrząsu')
    axs[1].title.set_text('Trend')
    axs[2].title.set_text('Sezonowość wystąpienia wstrząsu')
    axs[3].title.set_text('Energia wstrząsów, które wystapiły i nie wykazywały sie sezonowością')

    axs[0].set_ylabel("J",loc='top',rotation='horizontal')
    axs[1].set_ylabel("J",loc='top',rotation='horizontal')
    axs[2].set_ylabel("J",loc='top',rotation='horizontal')
    axs[3].set_ylabel("J",loc='top',rotation='horizontal')
    
    if title:
        plt.suptitle(title)
    t = title.replace('/','|')
    plt.savefig(f"{t}.png")

Function for faster calculations of series for groups of datasets

In [17]:
def helper(args):
        return get_activity_time_series(*args)

def get_activity_time_series_from_groups(groups, t_min, t_max):

    with Pool(16) as p:
        serie = p.map(helper, list(zip(groups.values(), [t_min]* len(groups), [t_max]*len(groups))))
    return serie
        

Start and end of the dataset period

In [18]:
t_min = min(data.TIME)
t_max = max(data.TIME)

Analysis of data grouped by region

In [19]:
groups_by_region_grouping = data.groupby('REJON')
groups_by_region = {k:groups_by_region_grouping.get_group(k).sort_values('TIME').reset_index(drop=True) for k in groups_by_region_grouping.groups}

series_by_region = get_activity_time_series_from_groups(groups_by_region, t_min,t_max)

for group,series in zip(groups_by_region.keys(),series_by_region):
    create_stl_plot(series,group)

100%|██████████| 43825/43825 [00:00<00:00, 65807.55it/s]


Analysis of data grouped by branch

In [ ]:
groups_by_branch_grouping = data.groupby('ODDZIAL')
groups_by_branch = {k:groups_by_branch_grouping.get_group(k).sort_values('TIME').reset_index(drop=True) for k in groups_by_branch_grouping.groups}

series_by_branch = get_activity_time_series_from_groups(groups_by_branch, t_min,t_max)

for group,series in zip(groups_by_branch.keys(),series_by_branch):
    create_stl_plot(series,group)

Analysis of data grouped by area


In [29]:
groups_by_area_grouping = data.groupby('POLE')
groups_by_area = {k:groups_by_area_grouping.get_group(k).sort_values('TIME').reset_index(drop=True) for k in groups_by_area_grouping.groups}

series_by_area = get_activity_time_series_from_groups(groups_by_area, t_min,t_max)

for group,series in zip(groups_by_area.keys(),series_by_area):
    create_stl_plot(series,group)